In [61]:
import pandas as pd
url="https://drive.google.com/file/d/1wqCgWx0Q3k9k1NfnkuwOcKDh1oz1S014/view?usp=sharing"
# url="https://drive.google.com/file/d/1r83ob7VN7ajV58QfM4FLLxzGIHMJ3Hn2/view?usp=sharing"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, delimiter=';')
base = df # backup

df = df[["content", "sentiment"]]

In [62]:
print(df["sentiment"].value_counts())

sentiment
Positivo     80
Negativo     80
Reembolso    40
Melhorias    40
Dúvidas      40
Name: count, dtype: int64


In [56]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('portuguese')
swToRemove = ['não','nem','quando','qual','quem','como','estou','mas']
for i in swToRemove:
  stopwords.remove(i)

def rem_stopwords(text):
  words = word_tokenize(text.lower())
  return [w for w in words if w not in stopwords]

df.content = df.content.apply(lambda x: " ".join(rem_stopwords(x)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jesse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jesse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\jesse\AppData\Local\Temp\ipykernel_24356\274151821.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.content = df.content.apply(lambda x: " ".join(rem_stopwords(x)))


In [57]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

frases = df.content.tolist()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(frases)
vocabulario = vectorizer.get_feature_names_out()

frases_vec = vectorizer.transform(frases).toarray()
sentimentos = np.array(df['sentiment'])

In [58]:
from sklearn.model_selection import train_test_split
treino_x, teste_x, treino_y, teste_y = train_test_split(frases_vec, sentimentos, test_size= 0.3)

In [59]:
from sklearn.svm import SVC
modelo = SVC(kernel='sigmoid') # (kernel='linear')
modelo.fit(treino_x, treino_y)
y_pred = modelo.predict(teste_x)

In [60]:
# Salvar o modelo
import os
import joblib
if not os.path.exists('models'):
    os.makedirs('models')
model_path = os.path.join('models', 'rbf_modelo_treinado_svc.pkl')
joblib.dump(modelo, model_path)
print(f"Modelo salvo em {model_path}")

# Salvar o Vectorizer
if not os.path.exists('models'):
    os.makedirs('models')
vectorizer_path = os.path.join('models', 'rbf_vectorizer.pkl')
joblib.dump(vectorizer, vectorizer_path)
print(f"Vectorizer salvo em {vectorizer_path}")

# Salvar o stopwords
if not os.path.exists('models'):
    os.makedirs('models')
stopwords_path = os.path.join('models', 'rbf_stopwords.pkl')
joblib.dump(stopwords, stopwords_path)
print(f"Vectorizer salvo em {stopwords_path}")


Modelo salvo em models\rbf_modelo_treinado_svc.pkl
Vectorizer salvo em models\rbf_vectorizer.pkl
Vectorizer salvo em models\rbf_stopwords.pkl
